In [2]:
pip install surprise

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install docx

In [ ]:
from docx import Document

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
df_final = pd.read_csv("../data/df_final.csv") 

In [6]:
df_final.head()

,user_id,app_id,category,rating
0,1,1,Strategy,4
1,2,2,Puzzle,3
2,3,3,Business,2
3,4,4,Simulation,1
4,5,5,Books & Reference,3
